# Approach 1: Non-linear adaptation with AdapterMLP

 In this notebook a first non-linear approach is tested. We take all the activations of both LLMs, we train 3 classifiers (1 per type of layer) for the Teacher model, with an AdapterMLP we try to adapt the Student latent space to the Teacher one. Finally we test the adapted Student activations with the Teacher classifiers.

In [37]:
import json
import os
import numpy as np
import matplotlib.pyplot as plt
import gc
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
import traceback
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math

In [38]:

PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
CACHE_DIR_NAME = "activation_cache"
HF_DEFAULT_HOME = os.environ.get("HF_HOME", "~\\.cache\\huggingface\\hub")



# We test the same layers as in the linear approach
LAYER_CONFIG = {
    "Qwen2.5-7B": 
    {
        "attn": [15,16,18],
        "mlp":[16,18,20],
        "hidden": [18,19,20]
    },    
    "Falcon3-7B-Base": 
    {
        "attn": [2,7,12],
        "mlp":[10,11,12],
        "hidden": [2,3,19]
    }
}




### Dataset preparation

In [39]:
def stats_per_json(model_name, dataset_name):
    file_path = os.path.join(PROJECT_ROOT, CACHE_DIR_NAME, model_name, dataset_name,"generations","hallucination_labels.json")
    with open(file_path, 'r') as file:
        data = json.load(file)
    total = len(data)
    hallucinations = sum(1 for item in data if item['is_hallucination'])
    percent_hallucinations = (hallucinations / total) * 100 if total > 0 else 0
    allucinated_items = [item['instance_id'] for item in data if item['is_hallucination']]
    return {
        'total': total,
        'hallucinations': hallucinations,
        'percent_hallucinations': percent_hallucinations,
        'hallucinated_items': allucinated_items,
        'model_name': model_name,
        'dataset_name': dataset_name
    }


qwen_stats=stats_per_json("Qwen2.5-7B", "belief_bank")
falcon_stats=stats_per_json("Falcon3-7B-Base", "belief_bank")

In [ ]:
# ------------------------------------------------------------------
# 1. Dataset class
# ------------------------------------------------------------------
class AlignmentDataset(Dataset):
    def __init__(self, x_source: torch.Tensor, x_target: torch.Tensor):
        # Ora assumiamo che i dati siano già torch.Tensor
        self.x_source = x_source
        self.x_target = x_target
    
    def __len__(self):
        return self.x_source.shape[0]
    
    def __getitem__(self, idx):
        return self.x_source[idx], self.x_target[idx]

# ------------------------------------------------------------------
# 2. AlignmentNetwork
# ------------------------------------------------------------------
class AlignmentNetwork(nn.Module):
    def __init__(self, input_dim: int, output_dim: int, hidden_dim: int = 512, dropout: float = 0.3):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),   
            nn.GELU(),                          
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, output_dim),
        )
        
        # Shortcut Lineare (Impara la rotazione di base)
        if input_dim != output_dim:
            self.shortcut = nn.Linear(input_dim, output_dim, bias=False)
            with torch.no_grad():
                self.shortcut.weight.copy_(torch.eye(output_dim, input_dim) * 0.01)
        else:
            self.shortcut = nn.Identity()

        self._init_specific_weights()
    
    def _init_specific_weights(self):
        # Inizializza l'ultimo layer del non-lineare a zero
        # La rete parte ESATTAMENTE come lineare
        nn.init.zeros_(self.net[-1].weight)
        if self.net[-1].bias is not None:
            nn.init.zeros_(self.net[-1].bias)

    def forward(self, x, training=False):
        # Noise Injection solo in training
        if training:
            noise = torch.randn_like(x) * 0.05  # 5% di rumore aggiunto
            x = x + noise
            
        return self.shortcut(x) + self.net(x)
    



class MixedLoss(nn.Module):
    def __init__(self, alpha=0.01, beta=1.0):
        super().__init__()
        self.alpha = alpha  # Peso per MSE
        self.beta = beta    # Peso per Cosine
        self.mse = nn.MSELoss()

    def forward(self, pred, target):
        # 1. MSE Loss (Magnitudine e Posizione esatta)
        loss_mse = self.mse(pred, target)
        
        # 2. Cosine Loss (Direzione/Angolo)
        # Cosine Similarity restituisce valori tra -1 e 1.
        # Vogliamo massimizzare la similarità (1), quindi minimizziamo (1 - similarità).
        # Usiamo dim=1 per calcolare la similarità per ogni vettore nel batch.
        cosine_sim = F.cosine_similarity(pred, target, dim=1).mean()
        loss_cosine = 1 - cosine_sim
        
        # Loss combinata
        return self.alpha * loss_mse + self.beta * loss_cosine


    


In [41]:
def load_and_split_layers(model_name, dataset_name, layer_indices, type_layer, stats, train_indices, test_indices):
    """
    Caricamento standard in RAM (senza memmap).
    """
    print(f" Caricamento IN-MEMORY {model_name} [{type_layer}]: layers {layer_indices}...")

    total_samples = stats['total']
    hallucinated_set = set(stats['hallucinated_items'])

    # Label
    y_full = np.zeros(total_samples, dtype=np.int8)
    y_full[list(hallucinated_set)] = 1
    y_train = y_full[train_indices]
    y_test  = y_full[test_indices]

    # Load and concatenate
    all_features = []
    
    for layer_idx in layer_indices:
        file_path = os.path.join(PROJECT_ROOT, CACHE_DIR_NAME, model_name, dataset_name,
                                 "activation_"+type_layer, f"layer{layer_idx}_activations.pt")
        if not os.path.exists(file_path):
            print(f" Warning: Layer {layer_idx} non trovato. Salto.")
            continue

        print(f"  Loading layer {layer_idx}...", end=" ")
        acts = torch.load(file_path, map_location='cpu')
        
        if acts.shape[0] > total_samples:
            acts = acts[:total_samples]

        # Convert to numpy
        if isinstance(acts, torch.Tensor):
            X_layer = acts.float().numpy() 
        else:
            X_layer = acts.astype(np.float32)

        # Flatten
        if X_layer.ndim > 2:
            X_layer = X_layer.reshape(X_layer.shape[0], -1)
            
        all_features.append(X_layer)
        print(f"done ({X_layer.shape})")
        
        del acts
        gc.collect()

    if not all_features:
        raise ValueError(f"Nessun layer valido trovato per {model_name}")

    print(" Concatenating layers...")
    X_full = np.concatenate(all_features, axis=1)
    
    X_train = X_full[train_indices]
    X_test  = X_full[test_indices]
    
    print(f" Completato! Train: {X_train.shape}, Test: {X_test.shape}")

    return X_train, X_test, y_train, y_test


# ==================================================================
# 4. Pipeline 
# ==================================================================
def run_experiment_pipeline_cached(X_teacher, y_teacher, teacher_name,
                                   X_student, y_student, student_name, layer_type, config_name,
                                   patience=50, min_delta=1e-4):
    
    print(f"\n{'='*60}")
    print(f"EXPERIMENT: {layer_type.upper()} → {teacher_name} ← {student_name}")
    print(f"{'='*60}")

    # Dati già splittati (numpy per sklearn)
    X_A_train_full, X_A_test = X_teacher['X_train'], X_teacher['X_test']
    y_A_train_full, y_A_test = y_teacher['y_train'], y_teacher['y_test']
    X_B_train_full, X_B_test = X_student['X_train'], X_student['X_test']
    y_B_train_full, y_B_test = y_student['y_train'], y_student['y_test']

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # --------------------------------------------------
    # 1. Teacher Probing (su FULL training set)
    # --------------------------------------------------
    print("1. Training teacher probe on FULL training set...")
    probe_teacher = LogisticRegression(max_iter=1000, class_weight='balanced', solver='lbfgs', n_jobs=-1)
    probe_teacher.fit(X_A_train_full, y_A_train_full)
    
    # --- METRICHE TEACHER ---
    y_pred_teacher = probe_teacher.predict(X_A_test)
    cm_teacher = confusion_matrix(y_A_test, y_pred_teacher)
    acc_teacher = accuracy_score(y_A_test, y_pred_teacher)
    prec_teacher = precision_score(y_A_test, y_pred_teacher)
    rec_teacher = recall_score(y_A_test, y_pred_teacher)
    f1_teacher = f1_score(y_A_test, y_pred_teacher)
    print(f"   Acc teacher: {acc_teacher:.4f}")

    # --------------------------------------------------
    # 2. Alignment Training (Student → Teacher space)
    # --------------------------------------------------
    print("2. Training alignment network (with 90/10 validation split)...")
    
    # Preconversion a torch.Tensor UNA VOLTA SOLA
    X_A_train_full_t = torch.from_numpy(X_A_train_full).float()
    X_A_test_t       = torch.from_numpy(X_A_test).float()
    X_B_train_full_t = torch.from_numpy(X_B_train_full).float()
    X_B_test_t       = torch.from_numpy(X_B_test).float()

    # Create Validation Split (10%) per l'alignment network SOLTANTO
    num_train = len(X_B_train_full)
    indices = np.arange(num_train)
    np.random.seed(42)  # Set seed for reproducibility
    np.random.shuffle(indices)
    val_size = int(num_train * 0.1)
    train_indices = indices[val_size:]
    val_indices = indices[:val_size]

    # Slice diretta sui tensori (no conversione per-item)
    X_B_align_train = X_B_train_full_t[train_indices]
    X_A_align_train = X_A_train_full_t[train_indices]
    
    X_B_val = X_B_train_full_t[val_indices]
    X_A_val = X_A_train_full_t[val_indices]

    train_dataset = AlignmentDataset(X_B_align_train.to(device), X_A_align_train.to(device))
    val_dataset   = AlignmentDataset(X_B_val.to(device),  X_A_val.to(device))
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0, pin_memory=False)
    val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False, num_workers=0, pin_memory=False)
    
    criterion = MixedLoss().to(device)


    aligner = AlignmentNetwork(
        input_dim=X_B_align_train.shape[1],
        output_dim=X_A_align_train.shape[1],
    ).to(device)
    
    optimizer = optim.AdamW(aligner.parameters(), lr=1e-3, weight_decay=0.1)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)
    
    epochs = 1000
    
    # Early Stopping variables
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    
    for epoch in range(epochs):
        # Training
        aligner.train()
        epoch_loss = 0.0
        for data, target in train_loader:
            optimizer.zero_grad()
            projected = aligner(data)
            #projected = aligner(data, training=True)
            loss = loss = criterion(projected, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(aligner.parameters(), max_norm=1.0)  #avoid exploding gradients
            optimizer.step()
            epoch_loss += loss.item()
        
        avg_train_loss = epoch_loss / len(train_loader)
        
        # Validation
        aligner.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data, target in val_loader:
                projected = aligner(data)
                loss = loss = criterion(projected, target)
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader)
        
        scheduler.step()
        
        
        print(f"   Epoch {epoch+1:2d}/{epochs} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")
            
        # Early Stopping Check
        if avg_val_loss < best_val_loss - min_delta:
            best_val_loss = avg_val_loss
            patience_counter = 0
            best_model_state = aligner.state_dict()
        else:
            patience_counter += 1
            
        if patience_counter >= patience:
            print(f"   Early stopping triggered at epoch {epoch+1}. Best Val Loss: {best_val_loss:.6f}")
            break
            
    # Load best model
    if best_model_state is not None:
        aligner.load_state_dict(best_model_state)
    
    # Save the best alignment network to disk
    model_save_dir = os.path.join("alignment_models", layer_type)
    os.makedirs(model_save_dir, exist_ok=True)
    model_filename = os.path.join(model_save_dir, f"{config_name}_aligner_{student_name}_to_{teacher_name}.pt")
    
    torch.save({
        'model_state_dict': aligner.state_dict(),
        'input_dim': X_B_align_train.shape[1],
        'output_dim': X_A_align_train.shape[1],
        'dropout': 0.1,
        'best_val_loss': best_val_loss,
        'layer_type': layer_type,
        'student_model': student_name,
        'teacher_model': teacher_name,
    }, model_filename)
    print(f"   ✓ Alignment network saved: {model_filename}")

    # --------------------------------------------------
    # 3. Evaluation: Student projected → Teacher probe
    # --------------------------------------------------
    print("3. Projecting student test set & evaluating...")
    aligner.eval()
    with torch.no_grad():
        X_B_projected = aligner(X_B_test_t.to(device)).cpu().numpy()
    
    y_pred_cross = probe_teacher.predict(X_B_projected)
    
    # --- METRICHE CROSS-MODEL ---
    cm_cross = confusion_matrix(y_B_test, y_pred_cross)
    acc_cross = accuracy_score(y_B_test, y_pred_cross)
    prec_cross = precision_score(y_B_test, y_pred_cross)
    rec_cross = recall_score(y_B_test, y_pred_cross)
    f1_cross = f1_score(y_B_test, y_pred_cross)
    
    print(f"\nFINAL RESULT:")
    print(f"   Teacher Acc         : {acc_teacher:.4f}")
    print(f"   Student → Teacher F1: {acc_cross:.4f}")
    print(f"   Transfer gap       : {acc_teacher - acc_cross:.4f}")

    return {
        "type": layer_type,
        "teacher_name": teacher_name,
        "student_name": student_name,
        "teacher": {
            "accuracy": acc_teacher,
            "precision": prec_teacher,
            "recall": rec_teacher,
            "f1": f1_teacher,
            "confusion_matrix": cm_teacher.tolist()
        },
        "student_on_teacher": {
            "accuracy": acc_cross,
            "precision": prec_cross,
            "recall": rec_cross,
            "f1": f1_cross,
            "confusion_matrix": cm_cross.tolist()
        }
    }


def plot_confusion_matrix(cm, layer_type, model_name="", save_dir="confusion_matrices"):
    """
    Plotta e salva la confusion matrix come immagine.
    """
    os.makedirs(save_dir, exist_ok=True)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True, ax=ax,
                xticklabels=['Non-Hallucinated', 'Hallucinated'],
                yticklabels=['Non-Hallucinated', 'Hallucinated'])
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')
    title = f'Confusion Matrix - {layer_type.upper()} Layers'
    if model_name:
        title += f' ({model_name})'
    ax.set_title(title)
    
    plt.tight_layout()
    filename = os.path.join(save_dir, f'confusion_matrix_{layer_type}_{model_name}.png' if model_name else f'confusion_matrix_{layer_type}.png')
    plt.savefig(filename, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"   ✓ Salvato: {filename}")

In [42]:
print("="*80)
print("FASE 1: PRE-CARICAMENTO E SPLITTING DEI DATI (stessi indici shuffled per TUTTI i layer type)")
print("="*80 + "\n")

n_samples = qwen_stats['total'] 
rng = np.random.RandomState(42)
shuffled_indices = rng.permutation(n_samples)
split_idx = int(0.7 * n_samples)

train_indices = shuffled_indices[:split_idx]
test_indices = shuffled_indices[split_idx:]

# Definisci gli scenari di esperimento
scenarios = [
    {"teacher_model": "Qwen2.5-7B", "student_model": "Falcon3-7B-Base"},
    {"teacher_model": "Falcon3-7B-Base", "student_model": "Qwen2.5-7B"}
]

# Struttura per raccogliere i risultati mantenendo l'ordine degli scenari
scenario_results_map = {0: [], 1: []}

# Loop sui layer types (Carica -> Esegui -> Libera Memoria)
for layer_type in ['attn', 'mlp', 'hidden']:
    print(f"\n{'='*40}")
    print(f"PROCESSING LAYER TYPE: {layer_type.upper()}")
    print(f"{'='*40}")
    gc.collect()
    
    try:
        # 1. CARICAMENTO E SPLITTING STANDARD
        X_qwen_train, X_qwen_test, y_qwen_train, y_qwen_test = load_and_split_layers(
            "Qwen2.5-7B", "belief_bank", 
            LAYER_CONFIG["Qwen2.5-7B"][layer_type], 
            layer_type, qwen_stats,
            train_indices, test_indices
        )


        X_falcon_train, X_falcon_test, y_falcon_train, y_falcon_test = load_and_split_layers(
            "Falcon3-7B-Base", "belief_bank", 
            LAYER_CONFIG["Falcon3-7B-Base"][layer_type], 
            layer_type, falcon_stats,
            train_indices, test_indices
        )
        
        # 2. SCALING (con cast esplicito a float32 per risparmiare memoria)
        print("   Normalizzazione dati...")
        scaler_qwen = StandardScaler()
        X_qwen_train = scaler_qwen.fit_transform(X_qwen_train)
        X_qwen_test = scaler_qwen.transform(X_qwen_test)
        
        scaler_falcon = StandardScaler()
        X_falcon_train = scaler_falcon.fit_transform(X_falcon_train)
        X_falcon_test = scaler_falcon.transform(X_falcon_test)
        
        # Organizza i dati per l'uso
        current_data = {
            "qwen": {"X_train": X_qwen_train, "X_test": X_qwen_test, "y_train": y_qwen_train, "y_test": y_qwen_test},
            "falcon": {"X_train": X_falcon_train, "X_test": X_falcon_test, "y_train": y_falcon_train, "y_test": y_falcon_test}
        }

        # 3. ESECUZIONE ESPERIMENTI PER ENTRAMBI GLI SCENARI
        for i, scenario in enumerate(scenarios):
            print(f"\n   --- Scenario: {scenario['teacher_model']} -> {scenario['student_model']} ---")
            
            if scenario['teacher_model'] == "Qwen2.5-7B":
                X_teacher_data = current_data['qwen']
                X_student_data = current_data['falcon']
            else:
                X_teacher_data = current_data['falcon']
                X_student_data = current_data['qwen']
            
            res = run_experiment_pipeline_cached(
                X_teacher_data, X_teacher_data, scenario['teacher_model'],
                X_student_data, X_student_data, scenario['student_model'],
                layer_type, "CONFIG1"
            )
            scenario_results_map[i].append(res)
            
            # Plot confusion matrices
            plot_confusion_matrix(
                np.array(res['teacher']['confusion_matrix']), 
                layer_type, 
                f"Teacher_{scenario['teacher_model'].split('.')[0]}"
            )
            plot_confusion_matrix(
                np.array(res['student_on_teacher']['confusion_matrix']), 
                layer_type, 
                f"{scenario['student_model'].split('.')[0]}_on_{scenario['teacher_model'].split('.')[0]}"
            )

        # 4. PULIZIA MEMORIA
        del current_data, X_qwen_train, X_qwen_test, X_falcon_train, X_falcon_test
        del scaler_qwen, scaler_falcon
        gc.collect()
        torch.cuda.empty_cache()
        print(f"   Memoria liberata per {layer_type}.")

    except Exception as e:
        print(f"Errore critico nel layer {layer_type}: {e}")
        traceback.print_exc()
        exit(1)

# Ricostruisci la struttura all_results per il salvataggio JSON
all_results = []
for i, scenario in enumerate(scenarios):
    all_results.append({
        "scenario": f"{scenario['teacher_model']} (teacher) → {scenario['student_model']} (student)",
        "results": scenario_results_map[i]
    })

# Salva tutti i risultati in JSON
os.makedirs("results_metrics", exist_ok=True)
metrics_file = "results_metrics/experiment_results_all_scenarios.json"

all_results_json = []
for scenario_data in all_results:
    scenario_results = []
    for r in scenario_data['results']:
        scenario_results.append({
            "layer_type": r['type'],
            "teacher_model": r['teacher_name'],
            "student_model": r['student_name'],
            "teacher": {
                "accuracy": round(r['teacher']['accuracy'], 4),
                "precision": round(r['teacher']['precision'], 4),
                "recall": round(r['teacher']['recall'], 4),
                "f1_score": round(r['teacher']['f1'], 4),
                "confusion_matrix": {
                    "TN": int(r['teacher']['confusion_matrix'][0][0]),
                    "FP": int(r['teacher']['confusion_matrix'][0][1]),
                    "FN": int(r['teacher']['confusion_matrix'][1][0]),
                    "TP": int(r['teacher']['confusion_matrix'][1][1])
                }
            },
            "student_on_teacher": {
                "accuracy": round(r['student_on_teacher']['accuracy'], 4),
                "precision": round(r['student_on_teacher']['precision'], 4),
                "recall": round(r['student_on_teacher']['recall'], 4),
                "f1_score": round(r['student_on_teacher']['f1'], 4),
                "confusion_matrix": {
                    "TN": int(r['student_on_teacher']['confusion_matrix'][0][0]),
                    "FP": int(r['student_on_teacher']['confusion_matrix'][0][1]),
                    "FN": int(r['student_on_teacher']['confusion_matrix'][1][0]),
                    "TP": int(r['student_on_teacher']['confusion_matrix'][1][1])
                }
            }
        })
    
    all_results_json.append({
        "scenario": scenario_data['scenario'],
        "results": scenario_results
    })

with open(metrics_file, 'w') as f:
    json.dump(all_results_json, f, indent=2)

print(f"\n✓ Risultati salvati in: {metrics_file}")


FASE 1: PRE-CARICAMENTO E SPLITTING DEI DATI (stessi indici shuffled per TUTTI i layer type)


PROCESSING LAYER TYPE: ATTN
 Caricamento IN-MEMORY Qwen2.5-7B [attn]: layers [15, 16, 18]...
  Loading layer 15... done ((27416, 3584))
  Loading layer 16... done ((27416, 3584))
  Loading layer 16... done ((27416, 3584))
  Loading layer 18... done ((27416, 3584))
  Loading layer 18... done ((27416, 3584))
 Concatenating layers...
done ((27416, 3584))
 Concatenating layers...
 Completato! Train: (19191, 10752), Test: (8225, 10752)
 Caricamento IN-MEMORY Falcon3-7B-Base [attn]: layers [2, 7, 12]...
  Loading layer 2...  Completato! Train: (19191, 10752), Test: (8225, 10752)
 Caricamento IN-MEMORY Falcon3-7B-Base [attn]: layers [2, 7, 12]...
  Loading layer 2... done ((27416, 3072))
  Loading layer 7... done ((27416, 3072))
  Loading layer 7... done ((27416, 3072))
  Loading layer 12... done ((27416, 3072))
  Loading layer 12... done ((27416, 3072))
 Concatenating layers...
done ((27416, 3072))